# How to build p-net

## Prepare environment and  define parameters 

In [ ]:
is_colab = False
import sys

if is_colab:
    !pip install -q torch==1.9.0
    !pip install -q torchvision==0.10.0
    !pip install -q qiskit==0.20.0
    !pip install qfnn
    !wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1w9VRv0iVfsH20Kb_MkF3yFhFeiYDVy5n' -O model.tar.gz
    !tar zxvf /content/model.tar.gz

import torch

from qiskit import  QuantumCircuit, ClassicalRegister
import functools


from qfnn.qf_fb.q_output import fire_ibmq,analyze,add_measure
from qfnn.qf_circ.p_lyr_circ import P_LYR_Circ,P_Neuron_Circ
from qfnn.qf_fb.c_input import load_data,to_quantum_matrix

from qfnn.qf_net.p_lyr import P_LYR
import torch.nn as nn
print = functools.partial(print, flush=True)


## Inference

### get parameters of the model

In [ ]:
################ Weiwen on 12-30-2020 ################
# Parameters of the trained model 
######################################################

# Model initialization
weight_1 = torch.tensor([[1.,  -1.,  1.,  1.],[-1., 1., 1., 1.]])
weight_2 = torch.tensor([[1.,  -1.],[-1.,  -1.]])
angle = [0,0,0,0]
norm_flag = [True,False]
norm_para = torch.tensor([0.3060,0.6940])



### build the network

In [ ]:
################ Weiwen on 12-30-2020 ################
# Generate the circuit of the first p-layer 
######################################################
#define your input and output number
input_list = []
aux_list = []
output_list = []
circuit = QuantumCircuit()
for i in range(2):
    #init circuit
    p_layer = P_Neuron_Circ(4) 
    #add input qubit to your circuit
    inps = p_layer.add_input_qubits(circuit,'p'+str(i)+"_input")
    input_list.append(inps)
    #add auxiliary qubit to your circuit
    aux =p_layer.add_aux(circuit,'aux'+str(i)+"_qubit")
    aux_list.append(aux)
    #add output qubit to your circuit
    output = p_layer.add_out_qubits(circuit,'p_out_'+str(i)+"_qubit")
    output_list.append(output)
    #add p-neuron to your circuit
    p_layer.forward(circuit,[weight_1[i]],input_list[i][0],output_list[i],angle,aux_list[i])

circuit.draw('text',300)



In [ ]:
################ Weiwen on 12-30-2020 ################
# the second p-layer
######################################################

circuit.barrier()
p_layer = P_LYR_Circ(2,2)
p_layer_output = p_layer.add_out_qubits(circuit)

p_layer.forward(circuit,weight_2,output_list,p_layer_output)

add_measure(circuit,p_layer_output,'reg')

print("Output layer created!")

circuit.draw('text',fold =300)

### simulation

In [ ]:
# result
qc_shots=8192
opt_counts = fire_ibmq(circuit,qc_shots,True)
(opt_mycount,bits) = analyze(opt_counts)
opt_class_prob=[]
for b in range(bits):
    opt_class_prob.append(float(opt_mycount[b])/qc_shots)
print("Simukation Result :",opt_class_prob)

### classical inference

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 =P_LYR(4, 2, bias=False)
        self.fc2 =P_LYR(2, 2, bias=False)
    def forward(self,x):
        x = self.fc1(x)
        x = self.fc2(x)
        return x

model = Net()

state_dict= model.state_dict()
state_dict["fc1.weight"] = weight_1
state_dict["fc2.weight"] = weight_2
model.load_state_dict(state_dict)

state = torch.tensor([[0,0,0,0]],dtype= torch.float)
output = model.forward(state)
print("classical inference Result :",output)